In [1214]:
# Import needed libraries

import altair as alt
import pandas as pd
import numpy as np

## HW 1 - There is a New York Times visualization in nyt_orig.png file. You should create your own, very similar or even the same as it is in the picture.
#### You can find data in GLB.Ts_dSST.csv file.

### Part 1: Data exploration and preparation

In [1215]:
# Read csv file and show info

df = pd.read_csv('./GLB.Ts_dSST.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 139 entries, ('Year', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'J-D', 'D-N', 'DJF', 'MAM', 'JJA') to ('2017', '.96', '1.11', '1.13', '.93', '.88', '.70', '.81', '.86', '.74', '.88', '.87', '.89', '.90', '.89', '.96', '.98', '.79')
Data columns (total 1 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Land-Ocean: Global Means  139 non-null    object
dtypes: object(1)
memory usage: 55.1+ KB


In [1216]:
# Show first entries.

df.head()

,,,,,,,,,,,,,,,,,,Land-Ocean: Global Means
Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,J-D,D-N,DJF,MAM,JJA,SON
1880,-.29,-.18,-.11,-.19,-.11,-.23,-.21,-.09,-.16,-.23,-.20,-.23,-.19,***,***,-.14,-.18,-.20
1881,-.15,-.17,.04,.04,.02,-.20,-.06,-.02,-.13,-.20,-.21,-.10,-.10,-.11,-.18,.03,-.09,-.18
1882,.15,.15,.04,-.18,-.16,-.26,-.20,-.05,-.10,-.24,-.16,-.24,-.10,-.09,.06,-.10,-.17,-.17
1883,-.31,-.39,-.13,-.17,-.20,-.12,-.08,-.15,-.20,-.14,-.22,-.16,-.19,-.20,-.31,-.16,-.12,-.19


In [1217]:
df = df.reset_index(level=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])
new_header = df.iloc[0] 
df = df[1:]
df.columns = new_header

In [1218]:
df.columns

Index(['Year', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep',
       'Oct', 'Nov', 'Dec', 'J-D', 'D-N', 'DJF', 'MAM', 'JJA', 'SON'],
      dtype='object', name=0)

In [1219]:
cols = df.columns.drop('Year')
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')
df['Date'] = df['Year'].apply(pd.to_datetime)

In [1220]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138 entries, 1 to 138
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Year    138 non-null    object        
 1   Jan     138 non-null    float64       
 2   Feb     138 non-null    float64       
 3   Mar     138 non-null    float64       
 4   Apr     138 non-null    float64       
 5   May     138 non-null    float64       
 6   Jun     138 non-null    float64       
 7   Jul     138 non-null    float64       
 8   Aug     138 non-null    float64       
 9   Sep     138 non-null    float64       
 10  Oct     138 non-null    float64       
 11  Nov     138 non-null    float64       
 12  Dec     138 non-null    float64       
 13  J-D     138 non-null    float64       
 14  D-N     137 non-null    float64       
 15  DJF     137 non-null    float64       
 16  MAM     138 non-null    float64       
 17  JJA     138 non-null    float64       
 18  SON     13

In [1221]:
df = df[['Year', 'Date', 'J-D']]
mean = df.loc[(df['Date'] >= pd.Timestamp('1880')) & (df['Date'] < pd.Timestamp('1900'))]['J-D'].mean()
df['Relative'] = df['J-D'] - mean

In [1222]:
df.head()

,Year,Date,J-D,Relative
1,1880,1880-01-01,-0.19,0.0385
2,1881,1881-01-01,-0.10,0.1285
3,1882,1882-01-01,-0.10,0.1285
4,1883,1883-01-01,-0.19,0.0385
5,1884,1884-01-01,-0.29,-0.0615


### Part 2: Visualization

In [1223]:
x_decades = ['1880', '1890', '1900', '1910', '1920','1930','1940','1950',
             '1960','1970', '1980', '1990', '2000', '2010']
x_decades = pd.to_datetime(x_decades)


chart = alt.Chart(
            df,
            title={
                "text": ['Annual Global Surface Temperature', 'Relative to Late 19th Century Average'],
            }
        ).mark_point(
            color='black',
            fill = '#f27529',
            strokeWidth=1,
            size = 70,
            opacity=1
        ).encode(
            x = alt.X(
                field='Date',
                title = None,
                type='temporal',
                axis=alt.Axis(
                    values = list(x_decades)
                ),
            ),
            y = alt.Y(
                field ='Relative',
                title = None,
                type='quantitative',
                axis=alt.Axis(
                    values = [-0.4, -0.2, 0.0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2],
                    labelExpr = 'datum.value > 0 ? "+" + datum.value + "°" : datum.value + "°"',
                ),
            ),  
        ).properties(
            width=800,
            height=500
        )


In [1224]:
line = alt.Chart(pd.DataFrame({'y': [0]})).mark_rule().encode(y='y')
text = alt.Chart(pd.DataFrame({'x': [2010], 'y': [0]})).mark_text(
    fontSize=12,
    font='Courier',
    text='HOTTER THAN THE\n1880-1899 AVERAGE\n\nCOLDER',
    lineBreak='\n',
    dx=260,
    dy=-28
).encode(
    x='x:T',
    y='y:Q'
)


alt.layer(line, chart, text).configure_view(
         stroke='transparent'
    ).configure_axis(
        domainWidth=0.8
    ).configure_title(
        anchor='start',
        dx = 50,
        dy = 75,
        fontSize=14,
        font='Courier',
    ).properties(
        width=850,
        height=500
    )


alt.LayerChart(...)